In [1]:
# deprecate 관련 경고는 무시하도록 설정
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import folium                              # 지도 기반 데이터 시각화
from geopy.distance import great_circle    # 좌표 간 거리를 km 단위로 변환

## 반경 1km(도보 10~15분) 내 환경이 좋은 아파트 Top10 입니다.

In [3]:
# 데이터 불러오기
tapts = pd.read_csv("아파트Top10.csv")
tapts

,아파트명,주소,lat,lon,공원,전철역,편의시설,경찰서,병원,score,소요시간
0,당산신동아파밀리에,서울특별시 영등포구 영신로33길 3,37.519805,126.900396,1.0,5.0,10.0,12.0,4.0,70,20분
1,포레나영등포센트럴,서울특별시 영등포구 국회대로50길 20,37.523206,126.905671,2.0,4.0,7.0,11.0,2.0,61,16분
2,서울역센트럴자이아파트,서울특별시 중구 만리재로 175,37.554609,126.963096,3.0,7.0,4.0,2.0,3.0,60,27분
3,금호어울림,서울특별시 영등포구 문래로 137,37.519108,126.897049,0.0,5.0,10.0,9.0,4.0,59,24분
4,순영웰라이빌아파트,서울특별시 영등포구 영등포로62길 42,37.516573,126.912953,2.0,4.0,7.0,6.0,3.0,56,23분
5,당산한양,서울특별시 영등포구 영신로 193,37.525463,126.899751,0.0,4.0,12.0,12.0,2.0,55,22분
6,아크로타워스퀘어,서울특별시 영등포구 국회대로54길 10,37.522935,126.908306,2.0,3.0,6.0,9.0,2.0,53,18분
7,당산SHVILLE,서울특별시 영등포구 영신로 183,37.524215,126.901534,1.0,4.0,8.0,12.0,1.0,52,21분
8,당산진로,서울특별시 영등포구 당산로 68,37.520654,126.896360,0.0,4.0,12.0,10.0,2.0,52,26분
9,창전현대홈타운,서울특별시 마포구 서강로 83,37.550795,126.931844,1.0,7.0,10.0,5.0,1.0,52,36분


In [4]:
parks = pd.read_csv("서울_공원_좌표.csv")
subways = pd.read_csv("전국_전철역_좌표.csv")
stores = pd.read_csv("서울_편의시설_좌표.csv")
police = pd.read_csv("서울_경찰서_좌표.csv")
hospitals = pd.read_csv("서울_병원_좌표.csv")

In [5]:
class CountByWGS84:    # WGS84 위경도 좌표 기반 데이터 집계 및 시각화
    def __init__(self, df, num, lat, lon, dist=1):
        self.df = df
        self.num = num
        self.lat = lat
        self.lon = lon
        self.dist = dist       
        self.zoom = 15

    def filter_by_rectangle(self):   # 사각 범위 내 데이터 필터링
        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist
        lon_min = self.lon - 0.015 * self.dist
        lon_max = self.lon + 0.015 * self.dist
        
        self.points = [[lat_min, lon_min], [lat_max, lon_max]]
        
        result = self.df.loc[
            (self.df['lat'] > lat_min) & (self.df['lat'] < lat_max) &
            (self.df['lon'] > lon_min) & (self.df['lon'] < lon_max) ]
        result.index = range(len(result))
        return result
        
    def filter_by_radius(self):   # 반경 범위 내 데이터 필터링
        tmp = self.filter_by_rectangle()
        center = (self.lat, self.lon)

        result = pd.DataFrame()
        for index, row in tmp.iterrows():
            # 개별 좌표 포인트
            point = (row['lat'], row['lon'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, tmp.iloc[index, :].to_frame().T])
        result.index = range(len(result))
        return result

    def plot_by_radius(self, df):
        """
        반경 범위 내 데이터 시각화
        """
        m = folium.Map(location=[self.lat, self.lon], zoom_start=self.zoom)
        
        folium.Marker(location=[self.lat, self.lon],     # 아파트 마커
                      tooltip=aptName, 
                      icon=folium.Icon(color='red', icon='home', icon_color='white')
                     ).add_to(m)
        
        for idx, row in df.iterrows():
            lat_ = row['lat']
            lon_ = row['lon']

            folium.Marker(location=[lat_, lon_],        # 마커들
                          icon=folium.Icon(color=color[self.num], icon=icon[self.num], icon_color='white', prefix='fa'), 
                          tooltip=row[colName[self.num]]).add_to(m)

        folium.Circle(radius=dist * 1000,
                      location=[lat, lon],
                      color="#ff7800",
                      fill_color='#ffff00',
                      fill_opacity=0.2
                      ).add_to(m)

        return m

## ▼ 아파트와 반경을 선택하세요.

In [6]:
aptName = "당산신동아파밀리에"
dist = 1

In [7]:
c = tapts.loc[tapts['아파트명'].str.contains(aptName, na=False)]
lat = c.iloc[0,2]
lon = c.iloc[0,3]

icon = ['tree', 'train', 'shopping-cart', 'check', 'plus']
color = ['green', 'blue', 'orange', 'darkgreen', 'darkblue']
colName = ['공원명', '역사명', '사업장명', '구분', '병원명']

In [8]:
cbw = CountByWGS84(parks, 0, lat, lon, dist)
result_radius = cbw.filter_by_radius()

In [9]:
print(f"""
아파트명 : {aptName}
기준 거리: {dist} km
반경 범위 내 데이터 필터링 결과: {len(result_radius):,} 건
    {result_radius}
""")


아파트명 : 당산신동아파밀리에
기준 거리: 1 km
반경 범위 내 데이터 필터링 결과: 1 건
           공원명                     주소        lat         lon
0  중마루근린공원  서울특별시 영등포구 영등포동2가 222  37.519832  126.911267



In [10]:
cbw.plot_by_radius(result_radius)

In [11]:
cbw = CountByWGS84(subways, 1, lat, lon, dist)
result_radius = cbw.filter_by_radius()
cbw.plot_by_radius(result_radius)

In [12]:
cbw = CountByWGS84(stores, 2, lat, lon, dist)
result_radius = cbw.filter_by_radius()
cbw.plot_by_radius(result_radius)

In [13]:
cbw = CountByWGS84(police, 3, lat, lon, dist)
result_radius = cbw.filter_by_radius()
cbw.plot_by_radius(result_radius)

In [14]:
cbw = CountByWGS84(hospitals, 4, lat, lon, dist)
result_radius = cbw.filter_by_radius()
cbw.plot_by_radius(result_radius)